In [36]:
import pandas as pd
import dask.dataframe as dd
from dask.diagnostics import ProgressBar


In [37]:
ddf_uground_shared = dd.read_parquet("output/uground_shared")
ddf_uground_multi = dd.read_parquet("output/uground_multi")


In [38]:
len(ddf_uground_shared), len(ddf_uground_multi)


(240075, 124701)

In [39]:
ddf_uground_multi.count().compute().max()


124694

In [40]:
ddf_uground_multi[ddf_uground_multi.대장_구분_코드.isna()].compute()


,전유부_용도,최저_지하_주택_전유부_층,지하_주택_전유부_수,층_용도,최저_지하_주택_층,지하_주택_층별개요_수,표제부_세대_수,표제부_가구_수,표제부_주_용도,표제부_기타_용도,...,에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,내진_설계_적용_여부,내진_능력
관리_건축물대장_PK,,,,,,,,,,,,,,,,,,,,,
11620-8327,02002,1,1.0,None,<NA>,NaN,<NA>,<NA>,None,None,...,None,NaN,<NA>,None,<NA>,None,<NA>,None,None,None
11620-9279,02003,1,1.0,None,<NA>,NaN,<NA>,<NA>,None,None,...,None,NaN,<NA>,None,<NA>,None,<NA>,None,None,None
11620-25217,02003,1,1.0,None,<NA>,NaN,<NA>,<NA>,None,None,...,None,NaN,<NA>,None,<NA>,None,<NA>,None,None,None
26140-13010,01001,1,1.0,None,<NA>,NaN,<NA>,<NA>,None,None,...,None,NaN,<NA>,None,<NA>,None,<NA>,None,None,None
44131-32795,02003,1,1.0,None,<NA>,NaN,<NA>,<NA>,None,None,...,None,NaN,<NA>,None,<NA>,None,<NA>,None,None,None
41430-2273,02003,1,2.0,None,<NA>,NaN,<NA>,<NA>,None,None,...,None,NaN,<NA>,None,<NA>,None,<NA>,None,None,None
28110-100178748,02003,1,2.0,None,<NA>,NaN,<NA>,<NA>,None,None,...,None,NaN,<NA>,None,<NA>,None,<NA>,None,None,None


공동주택에서 오류 7개는 제외

In [41]:
ddf_uground_shared[["시군구_코드", "법정동_코드"]].head()


,시군구_코드,법정동_코드
관리_건축물대장_PK,,
11110-100179949,11110,18300
11110-100192807,11110,18200
11110-100209165,11110,17500
11110-10710,11110,17500
11110-11031,11110,17400


In [42]:
ddf_uground_shared["시도_코드"] = ddf_uground_shared.시군구_코드.str[:2]
ddf_uground_shared["법정동8_코드"] = (
    ddf_uground_shared.시군구_코드 + ddf_uground_shared.법정동_코드.str[:3]
)
ddf_uground_shared["법정동10_코드"] = ddf_uground_shared.시군구_코드 + ddf_uground_shared.법정동_코드

with pd.option_context("display.max_rows", 10, "display.max_columns", 100):
    display(ddf_uground_shared.head())


,층_용도,최저_지하_주택_층,지하_주택_층별개요_수,표제부_세대_수,표제부_가구_수,표제부_주_용도,표제부_기타_용도,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,내진_설계_적용_여부,내진_능력,시도_코드,법정동8_코드,법정동10_코드
관리_건축물대장_PK,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
11110-100179949,None,<NA>,NaN,0,4,01000,"다가구주택,근린생활시설",1,일반,2,일반건축물,서울특별시 종로구 평창동 182-1번지,서울특별시 종로구 평창12길 13,None,11110,18300,0,0182,0001,None,None,None,0,111104100488,18301,0,13,<NA>,None,0,주건축물,169.00,100.04,59.20,490.91,332.77,196.91,21,철근콘크리트구조,철근콘크리트구조,01000,단독주택,"다가구주택,근린생활시설",10,(철근)콘크리트,(철근)콘크리트,0,4,13.500,4,1,0,0,0,0.0,490.91,0,0.0,0,0.0,0,0.0,5,0.0,20080109,20080211,20080701,2008,3000082,건축과,1101,신축허가,0,None,0.0,0,None,0,None,0,20190806,1,None,11,11110183,1111018300
11110-100192807,01003,1,3.0,0,10,01000,다가구주택,1,일반,2,일반건축물,서울특별시 종로구 구기동 55-17번지,서울특별시 종로구 홍지문길 88,None,11110,18200,0,0055,0017,None,None,None,0,111104100556,18201,0,88,0,None,0,주건축물,297.00,88.29,29.73,396.09,264.87,89.18,21,철근콘크리트구조,철근콘크리트구조,01000,단독주택,다가구주택,10,(철근)콘크리트,(철근)콘크리트,0,10,11.390,3,1,0,0,0,0.0,396.09,0,0.0,0,0.0,0,0.0,7,80.5,20120312,20120327,20120926,2012,3000082,건축과,1101,신축허가,0,None,0.0,0,None,0,None,0,20190530,1,None,11,11110182,1111018200
11110-100209165,None,<NA>,NaN,0,8,01000,단독주택(다가구주택),1,일반,2,일반건축물,서울특별시 종로구 숭인동 181-51번지,서울특별시 종로구 종로65길 42,갤러리,11110,17500,0,0181,0051,None,None,None,3,111104100370,17501,0,42,0,None,0,주건축물,232.16,127.47,54.91,599.58,456.73,196.73,21,철근콘크리트구조,철근콘크리트구조,01000,단독주택,단독주택(다가구주택),10,(철근)콘크리트,(철근)콘크리트,0,8,14.952,5,1,1,0,0,0.0,599.58,0,0.0,0,0.0,6,0.0,1,0.0,20160908,20170407,20180214,2016,3000082,건축과,1101,신축허가,0,None,0.0,0,None,0,None,0,20191119,1,None,11,11110175,1111017500
11110-10710,01003,1,1.0,0,5,01000,다가구용주택(5가구),1,일반,2,일반건축물,서울특별시 종로구 숭인동 254번지,서울특별시 종로구 종로58길 8-3,None,11110,17500,0,0254,0000,None,None,None,0,111104100359,17501,0,8,3,None,0,주건축물,181.80,0.00,0.00,296.19,0.00,0.00,11,벽돌구조,연와조,01000,단독주택,다가구용주택(5가구),10,(철근)콘크리트,평스라브,0,5,0.000,2,1,0,0,0,0.0,296.19,0,0.0,0,0.0,0,0.0,0,0.0,19920813,None,19921128,None,None,None,None,None,0,None,0.0,0,None,0,None,0,20200427,0,None,11,11110175,1111017500
11110-11031,01003,1,1.0,0,10,01000,다가구용단독주택(10),1,일반,2,일반건축물,서울특별시 종로구 창신동 23-520번지,서울특별시 종로구 창신11길 12,None,11110,17400,0,0023,0520,None,None,None,0,111104100431,17401,0,12,0,None,0,주건축물,179.50,102.17,56.91,406.18,0.00,155.04,11,벽돌구조,연와조,01000,단독주택,다가구용단독주택(10),10,(철근)콘크리트,평스라브,0,10,8.750,3,1,0,0,0,0.0,406.18,0,0.0,0,0.0,0,0.0,3,34.5,19940728,19940808,19941215,None,None,None,None,None,0,None,0.0,0,None,0,None,0,20200427,0,None,11,11110174,1111017400


In [43]:
ddf_uground_shared_count = ddf_uground_shared[
    ["대장_구분_코드", "시군구_코드", "법정동_코드", "시도_코드", "법정동8_코드", "법정동10_코드"]
]


for group, suffix in zip(
    [["시도_코드"], ["시군구_코드"], ["시군구_코드", "법정동_코드"], ["법정동8_코드"], ["법정동10_코드"]],
    ["sido", "sgg", "bjd", "bjd8", "bjd10"],
):
    ddf_uground_shared_count.groupby(group).대장_구분_코드.count().to_csv(
        f"output/uground/shared_count_{suffix}.csv",
        single_file=True,
        encoding="utf_8_sig",
    )


In [45]:
ddf_uground_multi["시도_코드"] = ddf_uground_multi.시군구_코드.str[:2]
ddf_uground_multi["법정동8_코드"] = (
    ddf_uground_multi.시군구_코드 + ddf_uground_multi.법정동_코드.str[:3]
)
ddf_uground_multi["법정동10_코드"] = ddf_uground_multi.시군구_코드 + ddf_uground_multi.법정동_코드


ddf_uground_multi_count = ddf_uground_multi[
    ["대장_구분_코드", "시군구_코드", "법정동_코드", "시도_코드", "법정동8_코드", "법정동10_코드"]
]


for group, suffix in zip(
    [["시도_코드"], ["시군구_코드"], ["시군구_코드", "법정동_코드"], ["법정동8_코드"], ["법정동10_코드"]],
    ["sido", "sgg", "bjd", "bjd8", "bjd10"],
):
    ddf_uground_multi_count.groupby(group).대장_구분_코드.count().to_csv(
        f"output/uground/multi_count_{suffix}.csv",
        single_file=True,
        encoding="utf_8_sig",
    )


In [52]:
df_uground_sum = pd.DataFrame()
df_uground_sum["shared"] = (
    ddf_uground_shared_count.groupby(group).대장_구분_코드.count().compute()
)
df_uground_sum["multi"] = (
    ddf_uground_multi_count.groupby(group).대장_구분_코드.count().compute()
)
df_uground_sum['sum'] = df_uground_sum.shared + df_uground_sum.multi
df_uground_sum = df_uground_sum.fillna(0).astype(int)
df_uground_sum


,shared,multi,sum
법정동10_코드,,,
1111010100,30,46,76
1111010200,8,61,69
1111013500,4,1,5
1111014000,38,6,44
1111014100,4,1,5
...,...,...,...
4785032031,1,0,0
4817041026,1,0,0
4824025024,1,1,2


In [53]:
for group, suffix in zip(
    [["시도_코드"], ["시군구_코드"], ["시군구_코드", "법정동_코드"], ["법정동8_코드"], ["법정동10_코드"]],
    ["sido", "sgg", "bjd", "bjd8", "bjd10"],
):

    df_uground_sum = pd.DataFrame()
    df_uground_sum["shared"] = (
        ddf_uground_shared_count.groupby(group).대장_구분_코드.count().compute()
    )
    df_uground_sum["multi"] = (
        ddf_uground_multi_count.groupby(group).대장_구분_코드.count().compute()
    )
    df_uground_sum["sum"] = df_uground_sum.shared + df_uground_sum.multi
    df_uground_sum = df_uground_sum.fillna(0).astype(int)
    df_uground_sum.to_csv(
        f"output/uground/sum_count_{suffix}.csv",
        encoding="utf_8_sig",
    )
